In [1]:
import sys
repo_path = '/home/micael.verissimo/paper_lzt/exp-connection-hep/'
sys.path.insert(0, repo_path)

In [2]:
import os

import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from src.callbacks import sp_index
from src.decorators import Summary
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

2025-07-09 17:11:26.690283: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-09 17:11:26.690324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-09 17:11:26.699042: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-09 17:11:26.769738: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-09 17:11:37.546644: W tensorflow/compiler/tf2

Welcome to JupyROOT 6.30/02


In [3]:
# load the data
sgn_df = pd.concat([pd.read_parquet(os.path.join(repo_path, f'data/processed/{iname}')) for iname in ['zee_avg250_100k.parquet']], axis=0)
bkg_df = pd.concat([pd.read_parquet(os.path.join(repo_path, f'data/processed/{iname}')) for iname in ['jf17_avg250_100k.parquet']], axis=0)

m_df = pd.concat([sgn_df, bkg_df], axis=0)
m_df = m_df.loc[m_df.cl_eta.abs() <= 2.5]
m_df = m_df.loc[m_df.cl_et >= 15000]


In [4]:
GeV = 1e3
#et_bins  = [3., 7., 10., 15., 20., 30., 40., 50., 1000000.]
et_bins  = [[15., 30], [30., 50.], [50., np.inf]]
eta_bins = [[0.0, 0.8], [0.8, 1.37], [1.37, 1.54], [1.54, 2.37], [2.37, 2.50]]
tot = 0
for iet, (l_iet, h_iet) in enumerate(et_bins):
    for ieta, (l_ieta, h_ieta) in enumerate(eta_bins):
        print(f'Processing et bin = [{l_iet}, {h_iet}[ and eta bin = [{l_ieta}, {h_ieta}[')
        m_df.loc[(m_df.cl_et >= l_iet*GeV) & (m_df.cl_et < h_iet*GeV), 'et_bin'] = iet
        m_df.loc[(m_df.cl_eta.abs() >= l_ieta) & (m_df.cl_eta.abs() < h_ieta), 'eta_bin'] = ieta
        l_samples = len(m_df.loc[(m_df.et_bin == iet) & (m_df.eta_bin == ieta)])
        print(f'Number of events in this bin: {l_samples}')
        #display(m_df.loc[(m_df.et_bin == iet) & (m_df.eta_bin == ieta), 'target'].value_counts().to_frame('Samples'))
        tot += l_samples

Processing et bin = [15.0, 30[ and eta bin = [0.0, 0.8[
Number of events in this bin: 37640
Processing et bin = [15.0, 30[ and eta bin = [0.8, 1.37[
Number of events in this bin: 26467
Processing et bin = [15.0, 30[ and eta bin = [1.37, 1.54[
Number of events in this bin: 6577
Processing et bin = [15.0, 30[ and eta bin = [1.54, 2.37[
Number of events in this bin: 42118
Processing et bin = [15.0, 30[ and eta bin = [2.37, 2.5[
Number of events in this bin: 4836
Processing et bin = [30.0, 50.0[ and eta bin = [0.0, 0.8[
Number of events in this bin: 38431
Processing et bin = [30.0, 50.0[ and eta bin = [0.8, 1.37[
Number of events in this bin: 28745
Processing et bin = [30.0, 50.0[ and eta bin = [1.37, 1.54[
Number of events in this bin: 6052
Processing et bin = [30.0, 50.0[ and eta bin = [1.54, 2.37[
Number of events in this bin: 23096
Processing et bin = [30.0, 50.0[ and eta bin = [2.37, 2.5[
Number of events in this bin: 3065
Processing et bin = [50.0, inf[ and eta bin = [0.0, 0.8[
Numbe

In [5]:
m_df.head()

,EventNumber,RunNumber,avgmu,cl_deta,cl_dphi,cl_e,cl_e0,cl_e1,cl_e2,cl_e233,...,mc_e_float,mc_et_float,mc_eta_float,mc_pdgid_float,mc_phi_float,seed_eta,seed_phi,target,et_bin,eta_bin
1,11106,0,227.0,0.2,0.2,36484.445312,-296.273865,19247.931641,17715.183594,12524.038086,...,0.0,0.0,0.0,0.0,0.0,1.023612,-1.333346,1,0.0,1.0
5,11157,0,255.0,0.2,0.2,39841.964844,-802.195190,28068.957031,13365.142578,19251.869141,...,0.0,0.0,0.0,0.0,0.0,1.454018,-1.822873,1,0.0,2.0
6,11158,0,285.0,0.2,0.2,179719.515625,0.000000,16250.171875,163441.250000,128326.375000,...,0.0,0.0,0.0,0.0,0.0,2.275902,3.140360,1,1.0,3.0
10,11125,0,239.0,0.2,0.2,39832.589844,-1573.268677,31329.984375,10756.395508,9683.039062,...,0.0,0.0,0.0,0.0,0.0,1.226709,-1.997884,1,0.0,1.0
14,11146,0,244.0,0.2,0.2,98000.617188,91.565575,58963.347656,39396.113281,34765.378906,...,0.0,0.0,0.0,0.0,0.0,-1.662684,0.708370,1,1.0,3.0


In [6]:
import itertools
import json
for iet_bin, ieta_bin in itertools.product(range(len(et_bins)), range(len(eta_bins))):
    print(f'Creating reference for et_bin={iet_bin} and eta_bin={ieta_bin}')
    references = {}
    aux_df = m_df.loc[(m_df.et_bin == iet_bin) & (m_df.eta_bin == ieta_bin)]
    for iop, target_eff in zip(['tight', 'medium', 'loose'], [0.90, 0.95, 0.99]):
        key = iop
        references[key] = {}
        references[key]['det'] = {}
        references[key]['det']['passed'] = np.ceil(target_eff*len(aux_df.loc[aux_df.target == 1]))
        references[key]['det']['total'] = len(aux_df.loc[aux_df.target == 1])
    with open(os.path.join(repo_path, f'data/processed/references_et{iet_bin}_eta{ieta_bin}.json'), 'w') as f:
        json.dump(references, f, indent=2)

Creating reference for et_bin=0 and eta_bin=0


Creating reference for et_bin=0 and eta_bin=1
Creating reference for et_bin=0 and eta_bin=2
Creating reference for et_bin=0 and eta_bin=3
Creating reference for et_bin=0 and eta_bin=4
Creating reference for et_bin=1 and eta_bin=0
Creating reference for et_bin=1 and eta_bin=1
Creating reference for et_bin=1 and eta_bin=2
Creating reference for et_bin=1 and eta_bin=3
Creating reference for et_bin=1 and eta_bin=4
Creating reference for et_bin=2 and eta_bin=0
Creating reference for et_bin=2 and eta_bin=1
Creating reference for et_bin=2 and eta_bin=2
Creating reference for et_bin=2 and eta_bin=3
Creating reference for et_bin=2 and eta_bin=4


In [ ]:
from scipy import optimize
import pandas as pd


class SequentialCuts:
    """
    A class to create sequential cuts on three variables to achieve a target efficiency for a given class.
    
    This class optimizes cut thresholds on three variables to maintain a specified efficiency level
    while maximizing background rejection.
    """
    
    def __init__(self, target_efficiency=0.9, optimization_method='nelder-mead', verbose=True):
        """
        Initialize the SequentialCuts class.
        
        Args:
            target_efficiency (float): Target efficiency for the signal class (0-1).
            optimization_method (str): Optimization method for finding cuts ('nelder-mead', 'powell', 'bfgs').
            verbose (bool): Whether to print optimization progress.
        """
        self.target_efficiency = target_efficiency
        self.optimization_method = optimization_method
        self.verbose = verbose
        self.cuts = None
        self.efficiency_achieved = None
        self.background_rejection = None
        self.variable_names = None
        
    def _calculate_efficiency(self, df, label_column, cuts, variable_names, signal_class=1):
        """
        Calculate efficiency for given cuts.
        
        Args:
            df (pd.DataFrame): Input DataFrame.
            label_column (str): Name of the label column.
            cuts (tuple): Tuple of three cut values (cut1, cut2, cut3).
            variable_names (list): List of three variable names to apply cuts on.
            signal_class (int): Label value for signal class.
            
        Returns:
            float: Efficiency for the signal class.
        """
        if len(variable_names) != 3:
            raise ValueError("Must provide exactly 3 variable names")
            
        # Apply cuts sequentially
        mask = (df[variable_names[0]] > cuts[0]) & \
               (df[variable_names[1]] > cuts[1]) & \
               (df[variable_names[2]] > cuts[2])
        
        # Calculate efficiency for signal class
        signal_mask = df[label_column] == signal_class
        signal_total = signal_mask.sum()
        
        if signal_total == 0:
            return 0.0
            
        signal_passed = (mask & signal_mask).sum()
        efficiency = signal_passed / signal_total
        
        return efficiency
    
    def _calculate_background_rejection(self, df, label_column, cuts, variable_names, signal_class=1):
        """
        Calculate background rejection for given cuts.
        
        Args:
            df (pd.DataFrame): Input DataFrame.
            label_column (str): Name of the label column.
            cuts (tuple): Tuple of three cut values.
            variable_names (list): List of three variable names.
            signal_class (int): Label value for signal class.
            
        Returns:
            float: Background rejection rate.
        """
        # Apply cuts
        mask = (df[variable_names[0]] > cuts[0]) & \
               (df[variable_names[1]] > cuts[1]) & \
               (df[variable_names[2]] > cuts[2])
        
        # Calculate background rejection
        background_mask = df[label_column] != signal_class
        background_total = background_mask.sum()
        
        if background_total == 0:
            return 1.0
            
        background_passed = (mask & background_mask).sum()
        background_rejection = 1.0 - (background_passed / background_total)
        
        return background_rejection
    
    def _objective_function(self, cuts, df, label_column, variable_names, signal_class=1):
        """
        Objective function to minimize. Minimizes the deviation from target efficiency
        while maximizing background rejection.
        
        Args:
            cuts (tuple): Tuple of three cut values.
            df (pd.DataFrame): Input DataFrame.
            label_column (str): Name of the label column.
            variable_names (list): List of three variable names.
            signal_class (int): Label value for signal class.
            
        Returns:
            float: Objective function value to minimize.
        """
        efficiency = self._calculate_efficiency(df, label_column, cuts, variable_names, signal_class)
        background_rejection = self._calculate_background_rejection(df, label_column, cuts, variable_names, signal_class)
        
        # Penalize deviation from target efficiency heavily
        efficiency_penalty = 1000 * (efficiency - self.target_efficiency) ** 2
        
        # Maximize background rejection (minimize negative rejection)
        rejection_reward = -background_rejection
        
        return efficiency_penalty + rejection_reward
    
    def optimize_cuts(self, df, variable_names, label_column, signal_class=1, initial_cuts=None):
        """
        Optimize cuts to achieve target efficiency.
        
        Args:
            df (pd.DataFrame): Input DataFrame.
            variable_names (list): List of three variable names to apply cuts on.
            label_column (str): Name of the label column.
            signal_class (int): Label value for signal class.
            initial_cuts (tuple): Initial guess for cuts. If None, uses data percentiles.
            
        Returns:
            tuple: Optimized cut values (cut1, cut2, cut3).
        """
        if len(variable_names) != 3:
            raise ValueError("Must provide exactly 3 variable names")
            
        # Check if columns exist
        missing_cols = [col for col in variable_names + [label_column] if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing columns in DataFrame: {missing_cols}")
            
        self.variable_names = variable_names
        
        if initial_cuts is None:
            # Use 25th percentile as initial guess for cuts
            initial_cuts = tuple(df[var].quantile(0.25) for var in variable_names)
        
        if self.verbose:
            print(f"Starting optimization with initial cuts: {dict(zip(variable_names, initial_cuts))}")
            print(f"Target efficiency: {self.target_efficiency}")
        
        # Optimize cuts
        result = optimize.minimize(
            self._objective_function,
            initial_cuts,
            args=(df, label_column, variable_names, signal_class),
            method=self.optimization_method,
            options={'maxiter': 1000}
        )
        
        if not result.success:
            print(f"Warning: Optimization did not converge: {result.message}")
        
        self.cuts = tuple(result.x)
        self.efficiency_achieved  = self._calculate_efficiency(df, label_column, self.cuts, variable_names, signal_class)
        self.background_rejection = self._calculate_background_rejection(df, label_column, self.cuts, variable_names, signal_class)
        
        if self.verbose:
            print(f"Optimized cuts: {dict(zip(variable_names, self.cuts))}")
            print(f"Achieved efficiency: {self.efficiency_achieved:.4f}")
            print(f"Background rejection: {self.background_rejection:.4f}")
            print(f"False Alarm: {1.0-self.background_rejection:.4f}")
        
        return self.cuts
    
    def apply_cuts(self, df):
        """
        Apply the optimized cuts to data.
        
        Args:
            df (pd.DataFrame): Input DataFrame.
            
        Returns:
            pd.Series: Boolean mask indicating which samples pass the cuts.
        """
        if self.cuts is None:
            raise ValueError("Cuts have not been optimized yet. Call optimize_cuts() first.")
            
        if self.variable_names is None:
            raise ValueError("Variable names not set. Call optimize_cuts() first.")
            
        # Check if columns exist
        missing_cols = [col for col in self.variable_names if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing columns in DataFrame: {missing_cols}")
            
        mask = (df[self.variable_names[0]] > self.cuts[0]) & \
               (df[self.variable_names[1]] > self.cuts[1]) & \
               (df[self.variable_names[2]] > self.cuts[2])
        return mask
    
    def evaluate_cuts(self, df, label_column, signal_class=1):
        """
        Evaluate the performance of the cuts on new data.
        
        Args:
            df (pd.DataFrame): Input DataFrame.
            label_column (str): Name of the label column.
            signal_class (int): Label value for signal class.
            
        Returns:
            dict: Dictionary containing efficiency, background rejection, and other metrics.
        """
        if self.cuts is None:
            raise ValueError("Cuts have not been optimized yet. Call optimize_cuts() first.")
            
        if self.variable_names is None:
            raise ValueError("Variable names not set. Call optimize_cuts() first.")
            
        mask = self.apply_cuts(df)
        
        # Calculate metrics
        efficiency = self._calculate_efficiency(df, label_column, self.cuts, self.variable_names, signal_class)
        background_rejection = self._calculate_background_rejection(df, label_column, self.cuts, self.variable_names, signal_class)
        
        # Calculate additional metrics
        signal_mask = df[label_column] == signal_class
        background_mask = df[label_column] != signal_class
        
        signal_total = signal_mask.sum()
        background_total = background_mask.sum()
        signal_passed = (mask & signal_mask).sum()
        background_passed = (mask & background_mask).sum()
        
        results = {
            'efficiency': efficiency,
            'background_rejection': background_rejection,
            'signal_total': signal_total,
            'signal_passed': signal_passed,
            'background_total': background_total,
            'background_passed': background_passed,
            'overall_acceptance': mask.sum() / len(mask),
            'cuts': dict(zip(self.variable_names, self.cuts))
        }
        
        return results

In [ ]:
m_cutbased_tight = SequentialCuts(target_efficiency=0.9, optimization_method='powell', verbose=True)
m_cutbased_tight.optimize_cuts(m_df[(m_df.et_bin == 1) & (m_df.eta_bin == 0)], 
                         variable_names=['cl_reta', 'cl_eratio', 'cl_rhad'], 
                         label_column='target', signal_class=1)

In [ ]:
m_cutbased_medium = SequentialCuts(target_efficiency=0.95, optimization_method='powell', verbose=True)
m_cutbased_medium.optimize_cuts(m_df[(m_df.et_bin == 1) & (m_df.eta_bin == 0)], 
                         variable_names=['cl_reta', 'cl_eratio', 'cl_rhad'], 
                         label_column='target', signal_class=1)

In [ ]:
m_cutbased_loose = SequentialCuts(target_efficiency=0.99, optimization_method='powell', verbose=True)
m_cutbased_loose.optimize_cuts(m_df[(m_df.et_bin == 1) & (m_df.eta_bin == 0)], 
                         variable_names=['cl_reta', 'cl_eratio', 'cl_rhad'], 
                         label_column='target', signal_class=1)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
import numpy as np


class NeuralNetworkClassifier:
    """
    A Neural Network classifier that can be tuned to achieve a target efficiency for a given class.
    
    This class trains a simple NN and finds the optimal threshold to achieve the target efficiency
    while maximizing background rejection.
    """
    
    def __init__(self, target_efficiency=0.9, hidden_layers=[64, 32], dropout_rate=0.2, 
                 learning_rate=0.001, epochs=100, batch_size=256, verbose=True):
        """
        Initialize the Neural Network classifier.
        
        Args:
            target_efficiency (float): Target efficiency for the signal class (0-1).
            hidden_layers (list): List of hidden layer sizes.
            dropout_rate (float): Dropout rate for regularization.
            learning_rate (float): Learning rate for Adam optimizer.
            epochs (int): Maximum number of training epochs.
            batch_size (int): Batch size for training.
            verbose (bool): Whether to print training progress.
        """
        self.target_efficiency = target_efficiency
        self.hidden_layers = hidden_layers
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        
        self.model = None
        self.scaler = None
        self.threshold = None
        self.efficiency_achieved = None
        self.background_rejection = None
        self.variable_names = None
        
    def _build_model(self, input_dim):
        """
        Build the neural network model.
        
        Args:
            input_dim (int): Number of input features.
            
        Returns:
            tf.keras.Model: Compiled neural network model.
        """
        model = Sequential()
        
        # Input layer
        model.add(Dense(self.hidden_layers[0], input_dim=input_dim, activation='relu'))
        model.add(Dropout(self.dropout_rate))
        
        # Hidden layers
        for layer_size in self.hidden_layers[1:]:
            model.add(Dense(layer_size, activation='relu'))
            model.add(Dropout(self.dropout_rate))
        
        # Output layer
        model.add(Dense(1, activation='sigmoid'))
        
        # Compile model
        model.compile(
            optimizer=Adam(learning_rate=self.learning_rate),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        return model
    
    def _find_optimal_threshold(self, y_true, y_pred_proba, signal_class=1):
        """
        Find the optimal threshold to achieve target efficiency.
        
        Args:
            y_true (np.array): True labels.
            y_pred_proba (np.array): Predicted probabilities.
            signal_class (int): Label value for signal class.
            
        Returns:
            float: Optimal threshold.
        """
        # Get ROC curve
        fpr, tpr, thresholds = roc_curve(y_true == signal_class, y_pred_proba)
        
        # Find threshold closest to target efficiency
        target_idx = np.argmin(np.abs(tpr - self.target_efficiency))
        optimal_threshold = thresholds[target_idx]
        
        achieved_efficiency = tpr[target_idx]
        background_rejection = 1 - fpr[target_idx]
        
        if self.verbose:
            print(f"Optimal threshold: {optimal_threshold:.4f}")
            print(f"Achieved efficiency: {achieved_efficiency:.4f}")
            print(f"Background rejection: {background_rejection:.4f}")
            print(f"False Alarm: {fpr[target_idx]:.4f}")
        
        return optimal_threshold, achieved_efficiency, background_rejection
    
    def train_and_optimize(self, df, variable_names, label_column, signal_class=1, 
                          test_size=0.2, validation_split=0.2, random_state=42):
        """
        Train the neural network and find optimal threshold.
        
        Args:
            df (pd.DataFrame): Input DataFrame.
            variable_names (list): List of variable names to use as features.
            label_column (str): Name of the label column.
            signal_class (int): Label value for signal class.
            test_size (float): Fraction of data to use for testing.
            validation_split (float): Fraction of training data to use for validation.
            random_state (int): Random state for reproducibility.
            
        Returns:
            dict: Training history and performance metrics.
        """
        # Check if columns exist
        missing_cols = [col for col in variable_names + [label_column] if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing columns in DataFrame: {missing_cols}")
        
        self.variable_names = variable_names
        
        # Prepare data
        X = df[variable_names].values
        y = (df[label_column] == signal_class).astype(int).values
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=y
        )
        
        # Scale features
        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        if self.verbose:
            print(f"Training on {len(variable_names)} features: {variable_names}")
            print(f"Training samples: {len(X_train)} (Signal: {y_train.sum()}, Background: {len(y_train) - y_train.sum()})")
            print(f"Test samples: {len(X_test)} (Signal: {y_test.sum()}, Background: {len(y_test) - y_test.sum()})")
        
        # Build and train model
        self.model = self._build_model(len(variable_names))
        
        # Early stopping callback
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1 if self.verbose else 0
        )
        
        # Train model
        history = self.model.fit(
            X_train_scaled, y_train,
            epochs=self.epochs,
            batch_size=self.batch_size,
            validation_split=validation_split,
            callbacks=[early_stopping],
            verbose=1 if self.verbose else 0
        )
        
        # Get predictions on test set
        y_pred_proba = self.model.predict(X_test_scaled, verbose=0).flatten()
        
        # Find optimal threshold
        self.threshold, self.efficiency_achieved, self.background_rejection = \
            self._find_optimal_threshold(y_test, y_pred_proba, signal_class)
        
        return {
            'history': history.history,
            'test_efficiency': self.efficiency_achieved,
            'test_background_rejection': self.background_rejection,
            'threshold': self.threshold
        }
    
    def predict_proba(self, df):
        """
        Predict probabilities for new data.
        
        Args:
            df (pd.DataFrame): Input DataFrame.
            
        Returns:
            np.array: Predicted probabilities.
        """
        if self.model is None:
            raise ValueError("Model has not been trained yet. Call train_and_optimize() first.")
        
        if self.scaler is None:
            raise ValueError("Scaler not fitted. Call train_and_optimize() first.")
        
        if self.variable_names is None:
            raise ValueError("Variable names not set. Call train_and_optimize() first.")
        
        # Check if columns exist
        missing_cols = [col for col in self.variable_names if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing columns in DataFrame: {missing_cols}")
        
        X = df[self.variable_names].values
        X_scaled = self.scaler.transform(X)
        
        return self.model.predict(X_scaled, verbose=0).flatten()
    
    def apply_cuts(self, df):
        """
        Apply the optimized threshold to data.
        
        Args:
            df (pd.DataFrame): Input DataFrame.
            
        Returns:
            pd.Series: Boolean mask indicating which samples pass the cuts.
        """
        if self.threshold is None:
            raise ValueError("Threshold has not been optimized yet. Call train_and_optimize() first.")
        
        probabilities = self.predict_proba(df)
        return probabilities > self.threshold
    
    def evaluate_cuts(self, df, label_column, signal_class=1):
        """
        Evaluate the performance of the model on new data.
        
        Args:
            df (pd.DataFrame): Input DataFrame.
            label_column (str): Name of the label column.
            signal_class (int): Label value for signal class.
            
        Returns:
            dict: Dictionary containing efficiency, background rejection, and other metrics.
        """
        if self.threshold is None:
            raise ValueError("Model has not been trained yet. Call train_and_optimize() first.")
        
        mask = self.apply_cuts(df)
        
        # Calculate metrics
        signal_mask = df[label_column] == signal_class
        background_mask = df[label_column] != signal_class
        
        signal_total = signal_mask.sum()
        background_total = background_mask.sum()
        signal_passed = (mask & signal_mask).sum()
        background_passed = (mask & background_mask).sum()
        
        efficiency = signal_passed / signal_total if signal_total > 0 else 0.0
        background_rejection = 1.0 - (background_passed / background_total) if background_total > 0 else 1.0
        
        results = {
            'efficiency': efficiency,
            'background_rejection': background_rejection,
            'false_alarm': 1.0 - background_rejection,
            'signal_total': signal_total,
            'signal_passed': signal_passed,
            'background_total': background_total,
            'background_passed': background_passed,
            'overall_acceptance': mask.sum() / len(mask),
            'threshold': self.threshold,
            'features_used': self.variable_names
        }
        
        return results

In [ ]:
# Example usage of Neural Network classifier
print("=== Neural Network Approach ===")
print()

# Create NN classifier for tight working point (90% efficiency)
nn_tight = NeuralNetworkClassifier(
    target_efficiency=0.9,
    hidden_layers=[5],
    dropout_rate=0,
    learning_rate=0.001,
    epochs=200,
    batch_size=256,
    verbose=True
)

# Train the model
print("Training NN for tight working point (90% efficiency)...")
train_results = nn_tight.train_and_optimize(
    df=m_df[(m_df.et_bin == 1) & (m_df.eta_bin == 0)],
    variable_names=['cl_reta', 'cl_eratio', 'cl_rhad'],
    label_column='target',
    signal_class=1,
    test_size=0.2,
    validation_split=0.2,
    random_state=42
)

print(f"\nTraining completed!")
print(f"Final test efficiency: {train_results['test_efficiency']:.4f}")
print(f"Final test background rejection: {train_results['test_background_rejection']:.4f}")
print(f"Optimal threshold: {train_results['threshold']:.4f}")

In [ ]:
# Create NN classifier for medium working point (95% efficiency)
print("\n" + "="*50)
print("Training NN for medium working point (95% efficiency)...")
nn_medium = NeuralNetworkClassifier(
    target_efficiency=0.95,
    hidden_layers=[5],
    dropout_rate=0.,
    learning_rate=0.001,
    epochs=200,
    batch_size=256,
    verbose=True
)

train_results_medium = nn_medium.train_and_optimize(
    df=m_df[(m_df.et_bin == 1) & (m_df.eta_bin == 0)],
    variable_names=['cl_reta', 'cl_eratio', 'cl_rhad'],
    label_column='target',
    signal_class=1,
    test_size=0.2,
    validation_split=0.2,
    random_state=42
)

print(f"Medium NN - Test efficiency: {train_results_medium['test_efficiency']:.4f}")
print(f"Medium NN - Test background rejection: {train_results_medium['test_background_rejection']:.4f}")

# Create NN classifier for loose working point (99% efficiency)
print("\n" + "="*50)
print("Training NN for loose working point (99% efficiency)...")
nn_loose = NeuralNetworkClassifier(
    target_efficiency=0.99,
    hidden_layers=[5],
    dropout_rate=0.,
    learning_rate=0.001,
    epochs=200,
    batch_size=256,
    verbose=True
)

train_results_loose = nn_loose.train_and_optimize(
    df=m_df[(m_df.et_bin == 1) & (m_df.eta_bin == 0)],
    variable_names=['cl_reta', 'cl_eratio', 'cl_rhad'],
    label_column='target',
    signal_class=1,
    test_size=0.2,
    validation_split=0.2,
    random_state=42
)

print(f"Loose NN - Test efficiency: {train_results_loose['test_efficiency']:.4f}")
print(f"Loose NN - Test background rejection: {train_results_loose['test_background_rejection']:.4f}")

In [ ]:
# Compare results between Sequential Cuts and Neural Network approaches
print("\n" + "="*60)
print("COMPARISON: Sequential Cuts vs Neural Network")
print("="*60)

test_data = m_df[(m_df.et_bin == 1) & (m_df.eta_bin == 0)]

print("\nTIGHT Working Point (90% target efficiency):")
print("-" * 45)
print(f"Sequential Cuts - Efficiency: {m_cutbased_tight.efficiency_achieved:.4f}, Background Rejection: {m_cutbased_tight.background_rejection:.4f}")
print(f"Neural Network  - Efficiency: {nn_tight.efficiency_achieved:.4f}, Background Rejection: {nn_tight.background_rejection:.4f}")

print("\nMEDIUM Working Point (95% target efficiency):")
print("-" * 45)
print(f"Sequential Cuts - Efficiency: {m_cutbased_medium.efficiency_achieved:.4f}, Background Rejection: {m_cutbased_medium.background_rejection:.4f}")
print(f"Neural Network  - Efficiency: {nn_medium.efficiency_achieved:.4f}, Background Rejection: {nn_medium.background_rejection:.4f}")

print("\nLOOSE Working Point (99% target efficiency):")
print("-" * 45)
print(f"Sequential Cuts - Efficiency: {m_cutbased_loose.efficiency_achieved:.4f}, Background Rejection: {m_cutbased_loose.background_rejection:.4f}")
print(f"Neural Network  - Efficiency: {nn_loose.efficiency_achieved:.4f}, Background Rejection: {nn_loose.background_rejection:.4f}")

# Evaluate on the same test data for fair comparison
print("\n" + "="*60)
print("EVALUATION ON TEST DATA")
print("="*60)

# Test Sequential Cuts
seq_tight_results = m_cutbased_tight.evaluate_cuts(test_data, 'target', signal_class=1)
seq_medium_results = m_cutbased_medium.evaluate_cuts(test_data, 'target', signal_class=1)
seq_loose_results = m_cutbased_loose.evaluate_cuts(test_data, 'target', signal_class=1)

# Test Neural Networks
nn_tight_results = nn_tight.evaluate_cuts(test_data, 'target', signal_class=1)
nn_medium_results = nn_medium.evaluate_cuts(test_data, 'target', signal_class=1)
nn_loose_results = nn_loose.evaluate_cuts(test_data, 'target', signal_class=1)

print(f"\nTIGHT - Sequential: Eff={seq_tight_results['efficiency']:.4f}, BkgRej={seq_tight_results['background_rejection']:.4f}")
print(f"TIGHT - Neural Net: Eff={nn_tight_results['efficiency']:.4f}, BkgRej={nn_tight_results['background_rejection']:.4f}")

print(f"\nMEDIUM - Sequential: Eff={seq_medium_results['efficiency']:.4f}, BkgRej={seq_medium_results['background_rejection']:.4f}")
print(f"MEDIUM - Neural Net: Eff={nn_medium_results['efficiency']:.4f}, BkgRej={nn_medium_results['background_rejection']:.4f}")

print(f"\nLOOSE - Sequential: Eff={seq_loose_results['efficiency']:.4f}, BkgRej={seq_loose_results['background_rejection']:.4f}")
print(f"LOOSE - Neural Net: Eff={nn_loose_results['efficiency']:.4f}, BkgRej={nn_loose_results['background_rejection']:.4f}")

In [ ]:
# Example usage of Neural Network classifier
print("=== Neural Network Approach ===")
print()

# Create NN classifier for tight working point (90% efficiency)
nn_tight = NeuralNetworkClassifier(
    target_efficiency=0.9,
    hidden_layers=[5],
    dropout_rate=0,
    learning_rate=0.001,
    epochs=200,
    batch_size=256,
    verbose=True
)

# Train the model
print("Training NN for tight working point (90% efficiency)...")
train_results = nn_tight.train_and_optimize(
    df=m_df[(m_df.et_bin == 1) & (m_df.eta_bin == 0)],
    variable_names=[f"cl_ring_{idx}" for idx in range(100)], #['cl_reta', 'cl_eratio', 'cl_rhad'],
    label_column='target',
    signal_class=1,
    test_size=0.2,
    validation_split=0.2,
    random_state=42
)

print(f"\nTraining completed!")
print(f"Final test efficiency: {train_results['test_efficiency']:.4f}")
print(f"Final test background rejection: {train_results['test_background_rejection']:.4f}")
print(f"Optimal threshold: {train_results['threshold']:.4f}")